In [7]:
with open("requirements.txt", "w") as f:
    f.write("kubernetes>=12.0.0\n")
    f.write("kfp==1.8.0\n")
    f.write("requests\n")
    
!pip install -r requirements.txt  --upgrade --user

  Using cached kubernetes-18.20.0-py2.py3-none-any.whl (1.6 MB)
     |████████████████████████████████| 62 kB 888 kB/s eta 0:00:011
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [5]:
import kfp.dsl as dsl
import kfp
from kfp import components

kfserving_op = components.load_component_from_url('https://raw.githubusercontent.com/kubeflow/pipelines/master/components/kubeflow/kfserving/component.yaml')

@dsl.pipeline(
  name='KFServing pipeline',
  description='A pipeline for KFServing.'
)
def kfservingPipeline(
    action='apply',
    model_name='tensorflow-sample',
    model_uri='gs://kfserving-samples/models/tensorflow/flowers',
    namespace='kubeflow-user-example-com',
    framework='tensorflow'):

    kfserving = kfserving_op(action = action,
                             model_name=model_name,
                             model_uri=model_uri,
                             namespace=namespace,
                             framework=framework).set_image_pull_policy('Always')



In [6]:
# Compile pipeline
kfp.compiler.Compiler().compile(kfservingPipeline, 'tf-flower.zip')

##### this is fixed in v1.3.1, so you are no longer to take actions
~note: please disable sidecar-injection on kubeflow-user namespace,
as knative-serving / knative-events are not enabled istio sidecar.
failed to disable could get RBAC error~


##### you have to REPLACE authervice_session token into yours. Go to Browser -> Developer Console -> Application -> Cookies
![title](img/cookies.png)

In [28]:
## the following example use python's request to send restapi requests
import base64
import json
import requests

with open('testdata.jpg', 'rb') as f:
    image_content = f.read()
    image_64_encode = base64.encodebytes(image_content).decode('utf-8')
headers = {"Cookie": "authservice_session=MTYzMDg5MjU5NnxOd3dBTkZKS1RWRk1VRkZGVHpWQk5Fa3pNMHhPTXpkV1draFVSMWN6U3pKVFFrVlJUMU5DU0ZoR1JGcFZXVFZaUmxnMFF6Tk9TVUU9fId9HVT3LD8wbAnYPmQ7v2k1-yLfGzNMBbAxKkcWh48-",
          "Host": "tensorflow-sample.kubeflow-user-example-com.example.com"}
payload = {"instances":[{"image_bytes": {"b64": image_64_encode}, "key": "1"}]}
resp = requests.post('http://istio-ingressgateway.istio-system/v1/models/tensorflow-sample:predict', headers=headers, data=json.dumps(payload))
print(resp.text)

{
    "predictions": [
        {
            "scores": [2.00651739e-05, 1.30275157e-05, 1.49190305e-06, 0.99996531, 1.13776073e-08, 1.72391111e-07],
            "prediction": 3,
            "key": "1"
        }
    ]
}


In [10]:
## the following example use curl to send restapi requests

!(echo -n '{"instances":[{"image_bytes": {"b64": "'; base64 testdata.jpg; echo '"}, "key": "1"}]}') | \
curl -XPOST -vvv -H "Host: tensorflow-sample.kubeflow-user-example-com.example.com" \
-H "Cookie: authservice_session=MTYzMDg5MjU5NnxOd3dBTkZKS1RWRk1VRkZGVHpWQk5Fa3pNMHhPTXpkV1draFVSMWN6U3pKVFFrVlJUMU5DU0ZoR1JGcFZXVFZaUmxnMFF6Tk9TVUU9fId9HVT3LD8wbAnYPmQ7v2k1-yLfGzNMBbAxKkcWh48-" \
-d @- \
http://istio-ingressgateway.istio-system/v1/models/tensorflow-sample:predict

Note: Unnecessary use of -X or --request, POST is already inferred.
*   Trying 10.108.129.129:80...
* TCP_NODELAY set
* Connected to istio-ingressgateway.istio-system (10.108.129.129) port 80 (#0)
> POST /v1/models/tensorflow-sample:predict HTTP/1.1
> Host: tensorflow-sample.kubeflow-user-example-com.example.com
> User-Agent: curl/7.68.0
> Accept: */*
> Cookie: authservice_session=MTYzMDg5MjU5NnxOd3dBTkZKS1RWRk1VRkZGVHpWQk5Fa3pNMHhPTXpkV1draFVSMWN6U3pKVFFrVlJUMU5DU0ZoR1JGcFZXVFZaUmxnMFF6Tk9TVUU9fId9HVT3LD8wbAnYPmQ7v2k1-yLfGzNMBbAxKkcWh48-
> Content-Length: 97252
> Content-Type: application/x-www-form-urlencoded
> Expect: 100-continue
> 
* Mark bundle as not supporting multiuse
< HTTP/1.1 100 Continue
* We are completely uploaded and fine
* Mark bundle as not supporting multiuse
< HTTP/1.1 200 OK
< content-length: 218
< content-type: application/json
< date: Mon, 06 Sep 2021 02:08:12 GMT
< server: envoy
< x-envoy-upstream-service-time: 1635
< 
{
    "predictions": [
        {
          